<a href="https://colab.research.google.com/github/ilona-krasnova/IRM3006/blob/main/IRM3006P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries in Python
import pandas as pd
import numpy as np
import re
import json

# install playwright
!pip install -q playwright
!playwright install

import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 27.0 MB/s eta 0:00:00
164.5 MiB [] 0% 0.0s164.5 MiB [] 0% 36.0s164.5 MiB [] 0% 29.0s164.5 MiB [] 0% 37.2s164.5 MiB [] 0% 35.5s164.5 MiB [] 0% 23.7s164.5 MiB [] 0% 20.4s164.5 MiB [] 0% 18.0s164.5 MiB [] 0% 17.1s164.5 MiB [] 0% 16.2s164.5 MiB [] 1% 15.2s164.5 MiB [] 1% 14.7s164.5 MiB [] 1% 14.4s164.5 MiB [] 1% 14.2s164.5 MiB [] 1% 13.6s164.5 MiB [] 1% 13.3s164.5 MiB [] 2% 13.1s164.5 MiB [] 2% 13.0s164.5 MiB [] 2% 12.6s164.5 MiB [] 2% 12.5s164.5 MiB [] 2% 12.3s164.5 MiB [] 2% 12.2s164.5 MiB [] 3% 12.0s164.5 MiB [] 3% 11.9s164.5 MiB [] 3% 11.7s164.5 MiB [] 3% 11.6s164.5 MiB [] 3% 11.4s164.5 MiB [] 3% 11.5s164.5 MiB [] 4% 11.3s164.5 MiB [] 4% 11.6s164.5 MiB [] 4% 11.5s164.5 MiB [] 4% 11.3s164.5 MiB [] 4% 11.1s164.5 MiB [] 4% 11.0s164.5 MiB [] 5% 10.9s164.5 MiB [] 5% 10.8s164.5 MiB [] 5% 10.7s164.5 MiB [] 5% 10.6s164.5 MiB [] 6% 10.5s164.5 MiB [] 6% 10.4s164.5 MiB [] 6% 10.3s164.5 MiB [] 6% 10.2s164.5 MiB [] 7% 10.1s164.5 MiB [] 7% 10.

In [ ]:
# Read search page and save to file

from playwright.async_api import Page

async def fetch_dynamic_content(url, browser):
    # Output "0" to console
    # print(0)

    # Create a new page within the existing browser context
    page = await browser.new_page()
    await page.goto(url)

    # Output "1" to console
    # print(1)

    # Wait for the dynamic content to load
    # await page.wait_for_selector('.result-item-text') # maybe find a better benchmark for page load?
    await page.wait_for_load_state('networkidle')

    # Output "2" to console
    # print(2)

    # Extract page content after rendering
    page_content = await page.content()

    # Write page content to "search_results.txt"
    with open("search_results.txt", "w") as f:
        f.write(page_content)

    # Close the page (but not the browser)
    await page.close()

    # Return the page content as a result
    return page_content

In [ ]:
# Process search results; output all article links

def get_article_links(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract href values and store them in an array
    href_values = [a['href'] for a in soup.select("h3.item-title > a") if 'href' in a.attrs]

    # `href_values` now contains all the href attributes as an array
    return href_values

# Load the content of `search_results.txt` into BeautifulSoup
# with open("search_results.txt", "r") as f:
#     html_content = f.read()

# href_values2 = get_article_links(html_content)

# # Print the array to the console with each entry on its own line
# for href in href_values2:
#     print(href)

In [ ]:
def find_dates_in_text(text):
    # If the text doesn't exist, return
    if not text:
        return []
    # Find all sequences of exactly 4 digits
    four_digit_numbers = re.findall(r'\d{4}', text)
    # Filter numbers within the range 1801 to 2025 and remove duplicates by converting to a set
    dates = {int(num) for num in four_digit_numbers if 1801 <= int(num) <= 2025}
    # Sort the unique dates in descending order
    return sorted(dates, reverse=True)

In [ ]:
# Process array of strings to remove duplicates and filter out unwanted values.
def filter_and_deduplicate_strings(strings):
    if not strings:
        return []
    unwanted_values = {'more', 'hide', 'Show All', 'Show Less', ''}
    # Remove duplicates, strip whitespace, and filter out unwanted values
    unique_strings = list({string.strip() for string in strings})
    filtered_strings = [text for text in unique_strings if text not in unwanted_values]
    return filtered_strings

In [ ]:
def read_details_segment(soup, segment_name):
    # Parse the HTML content with BeautifulSoup
    # soup = BeautifulSoup(content, 'html.parser')

    # Step 1: Find the span with the attribute data-details-label="title"
    title_span = soup.find('span', {'data-details-label': segment_name})

    if not title_span:
        print("No span with data-details-label='" + segment_name +"' found")
        return

    # Step 2: Find the div that is the parent of this span
    parent_div = title_span.find_parent('div')

    if not parent_div:
        print("No parent div found for the " + segment_name +" span")
        return

    # Step 3: Find the div that is the next sibling of the parent div
    next_div = parent_div.find_next_sibling('div')

    if not next_div:
        print("No next sibling div found for the parent div (" + segment_name + ")")
        return

    # Step 4: In this next div, find the span and get its contents
    target_span = next_div.find_all('span')

    # Step 5: Output the contents of the target span
    if not target_span:
        print("No spans found in the next div for " + segment_name)
        return []
    return [span.get_text() for span in target_span]

def read_details(content):
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Usage example with content
    title = read_details_segment(soup, 'title')
    # print(title[0])

    creator = filter_and_deduplicate_strings(read_details_segment(soup, 'creator'))
    # print(creator)

    ispartof = read_details_segment(soup, 'ispartof')
    # print(ispartof[0])

    publisher = read_details_segment(soup, 'publisher')
    # print(publisher[0])

    subject = filter_and_deduplicate_strings(read_details_segment(soup, 'subject'))
    # print(subject)

    description = read_details_segment(soup, 'description')
    # print(description[0])

    identifier = filter_and_deduplicate_strings(read_details_segment(soup, 'identifier'))
    # print(identifier)

    dates = find_dates_in_text(ispartof[0])

    # Construct JSON string with title, creator, etc.

    # Define your data as a dictionary
    json_data = {
        "title": title[0] if title else "",
        "creator": creator,
        "dates": dates,
        "ispartof": ispartof[0] if ispartof else "",
        "publisher": publisher[0] if publisher else "",
        "subject": subject,
        "description": description[0] if description else "",
        "identifier": identifier
    }

    # Convert the dictionary to a JSON string if needed
    json_string = json.dumps(json_data, ensure_ascii=False)

    print(json_string)

    # Append the JSON string to a text file
    with open("bibliometrics.txt", "a", encoding="utf-8") as file:
        file.write(json_string + "\n")  # Add newline for each JSON entry

# Main ♟

In [ ]:
async def main():
    async with async_playwright() as p:
        # Launch the browser once
        browser = await p.chromium.launch(headless=True)

        # Prompt user to input URL (OMNI Search)
        base_url = "https://ocul-crl.primo.exlibrisgroup.com/discovery/search?q=bombus+terrestris&vid=01OCUL_CRL%3ACRL_DEFAULT&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&mode=basic&displayMode=full&bulkSize=10&highlight=true&dum=true&query=any%2Ccontains%2Cbombus+terrestris&displayField=all&offset=0"

        # Truncate end of base_url after last "="
        truncate_position = base_url.rfind("=") + 1
        base_url = base_url[:truncate_position]

        # Prompt user to input first page number
        first_page = int(input("Enter first page number: "))

        # Prompt user to input last page number
        last_page = int(input("Enter last page number (incl.): "))

        # For each search result page
        for page in range(first_page -1, last_page):
            url = f"{base_url}{page*10}"
            print("Page ", page + 1)

            # Parse search results
            search_results = await fetch_dynamic_content(url, browser)

            # Parse HTML
            links = get_article_links(search_results)

            # For each article
            for link in links:
                print(link)
                content = await fetch_dynamic_content(link, browser)
                read_details(content)

        # # Fetch content from multiple URLs
        # content1 = await fetch_dynamic_content("https://ocul-crl.primo.exlibrisgroup.com/discovery/fulldisplay?docid=cdi_proquest_journals_2938293070&context=PC&vid=01OCUL_CRL:CRL_DEFAULT&lang=en&search_scope=NewDiscoveryNetwork&adaptor=Primo", browser)
        # content2 = await fetch_dynamic_content("https://ocul-crl.primo.exlibrisgroup.com/discovery/fulldisplay?docid=cdi_proquest_journals_2913337499&context=PC&vid=01OCUL_CRL:CRL_DEFAULT&lang=en&search_scope=NewDiscoveryNetwork&adaptor=Primo", browser)

        # # save content1 to file
        # with open("content1.txt", "w") as f:
        #     f.write(content1)

        # # save content2 to file
        # with open("content2.txt", "w") as f:
        #     f.write(content2)

        # Close the browser after all calls
        await browser.close()

# run main function
await main()

Enter first page number: 28
Enter last page number (incl.): 32
Page  28
Page  29
Page  30
Page  31
Page  32


In [ ]:
url = "https://ocul-crl.primo.exlibrisgroup.com/discovery/search?query=any,contains,bombus%20terrestris&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&vid=01OCUL_CRL:CRL_DEFAULT&mfacet=rtype,include,articles,1&mode=basic&offset=0"



https://ocul-crl.primo.exlibrisgroup.com/discovery/search?query=any,contains,bombus%20terrestris&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&vid=01OCUL_CRL:CRL_DEFAULT&mfacet=rtype,include,articles,1&mode=basic&offset=
